### Initialization

In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.ndimage import uniform_filter
import pandas as pd
import itables
from itables import show
import json
import time
from datetime import datetime, timedelta
import math
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import ee
import geemap
import warnings

from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from scipy import ndimage
from skimage import filters, measure, morphology, segmentation
import cv2

2025-09-12 18:55:03.451733: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Custome Modules
import custom_lib

In [3]:
# Check TensorFlow and GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.20.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


#### Google earth engine authentication

In [4]:
# Authenticate and initialize GEE
ee.Authenticate(auth_mode='notebook', scopes=[
    'https://www.googleapis.com/auth/earthengine',
    'https://www.googleapis.com/auth/drive'
])
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


### Declaration of Study Area

In [5]:
# Pulau Pari 
pulau_pari_coords = [
    [106.600, -5.840],  # Northeast corner
    [106.640, -5.840],  # Southeast corner  
    [106.640, -5.875],  # Southwest corner
    [106.600, -5.875],  # Northwest corner
    [106.600, -5.840]   # Close the polygon
]

### Variables that being used for Study

- Band List Documentation : https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites
- Coppernicus Sentinel-2 bands: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#bands

In [6]:
# Satellite Model Variables
optical_satellite_model = 'COPERNICUS/S2_SR_HARMONIZED'

    # Best model for sentinel 2 
    # COPERNICUS/S2_SR_HARMONIZED
    # Can also be used 
    # 'COPERNICUS/S2_SR'

    # Landsat 8 Model currently not yet explored and need to adjust the code
    # 'LANDSAT/LC08/C02/T1_L2'
    
sar_satellite_model = 'COPERNICUS/S1_GRD'
    # General Purpose 
    # 'COPERNICUS/S1_GRD' 

    # Better for Quantitave analysis
    # 'COPERNICUS/S1_GRD_FLOAT'

    # Raw SAR 1 Data, 
    # 'COPERNICUS/S1_SLC'


# Range of Satellite for samples
recent_date = ['2025-01-01', '2025-12-31']

# Cloud Coverage Percentage
cloud_coverage = 5
zoomed_var = 16

### Observed Area

In [7]:
custom_lib.study_location(pulau_pari_coords, optical_satellite_model, recent_date, cloud_coverage, zoomed_var)


 Study area Preview

 Study area coordinates: [[106.6, -5.84], [106.64, -5.84], [106.64, -5.875], [106.6, -5.875], [106.6, -5.84]]

 Study area picture date : 2025-01-01 to 2025-12-31


Map(center=[-5.857500171581422, 106.61999999999767], controls=(WidgetControl(options=['position', 'transparent…

### Check the data availability

In [8]:
sar_yearly_summary = custom_lib.check_sar_data_availability ( pulau_pari_coords, sar_satellite_model )

Checking  COPERNICUS/S1_GRD 's Data availability
Full date range: 2014-10-12 to 2025-09-08
Total years available: 10.9 years

Data Availability by Year:
2014:   4 images | Polrization Type: ['VV'] | Orbits: ['ASCENDING']
2015:  14 images | Polrization Type: ['VH', 'VV'] | Orbits: ['DESCENDING', 'ASCENDING']
2016:  26 images | Polrization Type: ['VH', 'VV'] | Orbits: ['DESCENDING', 'ASCENDING']
2017:  55 images | Polrization Type: ['VH', 'VV'] | Orbits: ['DESCENDING', 'ASCENDING']
2018:  58 images | Polrization Type: ['VH', 'VV'] | Orbits: ['DESCENDING', 'ASCENDING']
2019:  57 images | Polrization Type: ['VH', 'VV'] | Orbits: ['ASCENDING', 'DESCENDING']
2020:  49 images | Polrization Type: ['VH', 'VV'] | Orbits: ['ASCENDING', 'DESCENDING']
2021:  46 images | Polrization Type: ['VH', 'VV'] | Orbits: ['DESCENDING', 'ASCENDING']
2022:  52 images | Polrization Type: ['VH', 'VV'] | Orbits: ['ASCENDING', 'DESCENDING']
2023:  39 images | Polrization Type: ['VH', 'VV'] | Orbits: ['DESCENDING', 

In [9]:
sar_yearly_summary

[{'year': 2014, 'count': 4, 'polarizations': ['VV'], 'orbits': ['ASCENDING']},
 {'year': 2015,
  'count': 14,
  'polarizations': ['VH', 'VV'],
  'orbits': ['DESCENDING', 'ASCENDING']},
 {'year': 2016,
  'count': 26,
  'polarizations': ['VH', 'VV'],
  'orbits': ['DESCENDING', 'ASCENDING']},
 {'year': 2017,
  'count': 55,
  'polarizations': ['VH', 'VV'],
  'orbits': ['DESCENDING', 'ASCENDING']},
 {'year': 2018,
  'count': 58,
  'polarizations': ['VH', 'VV'],
  'orbits': ['DESCENDING', 'ASCENDING']},
 {'year': 2019,
  'count': 57,
  'polarizations': ['VH', 'VV'],
  'orbits': ['ASCENDING', 'DESCENDING']},
 {'year': 2020,
  'count': 49,
  'polarizations': ['VH', 'VV'],
  'orbits': ['ASCENDING', 'DESCENDING']},
 {'year': 2021,
  'count': 46,
  'polarizations': ['VH', 'VV'],
  'orbits': ['DESCENDING', 'ASCENDING']},
 {'year': 2022,
  'count': 52,
  'polarizations': ['VH', 'VV'],
  'orbits': ['ASCENDING', 'DESCENDING']},
 {'year': 2023,
  'count': 39,
  'polarizations': ['VH', 'VV'],
  'orbits

In [10]:
optical_yearly_summary = custom_lib.check_optical_data_availability ( pulau_pari_coords, optical_satellite_model )


Checking COPERNICUS/S2_SR_HARMONIZED's Optical Data Availability
Full date range: 2018-04-20 to 2025-09-10
Total years available: 7.4 years

Cloud Coverage Analysis:
Images with <10% clouds: 34
Images with <20% clouds: 78
Images with <30% clouds: 112
Images with <50% clouds: 178

Seasonal Availability (<20% clouds):
   Full month 2019-04-01 to 2019-04-30: 1 images
   Full month 2019-05-01 to 2019-05-31: 3 images
   Full month 2019-06-01 to 2019-06-30: 3 images
   Full month 2019-07-01 to 2019-07-31: 1 images
   Full month 2019-08-01 to 2019-08-31: 4 images
   Full month 2019-09-01 to 2019-09-30: 2 images
   Full month 2019-10-01 to 2019-10-31: 1 images
   Full month 2019-11-01 to 2019-11-30: 1 images
   Full month 2019-12-01 to 2019-12-31: 1 images
   Full month 2020-05-01 to 2020-05-31: 2 images
   Full month 2020-06-01 to 2020-06-30: 2 images
   Full month 2020-07-01 to 2020-07-31: 2 images
   Full month 2020-08-01 to 2020-08-31: 4 images
   Full month 2020-09-01 to 2020-09-30: 1 im

In [11]:
optical_yearly_summary

[{'year': 2018, 'dry_season_count': 0, 'wet_season_count': 1, 'count': 0},
 {'year': 2019, 'dry_season_count': 14, 'wet_season_count': 2, 'count': 17},
 {'year': 2020, 'dry_season_count': 11, 'wet_season_count': 3, 'count': 11},
 {'year': 2021, 'dry_season_count': 7, 'wet_season_count': 1, 'count': 10},
 {'year': 2022, 'dry_season_count': 1, 'wet_season_count': 1, 'count': 2},
 {'year': 2023, 'dry_season_count': 15, 'wet_season_count': 2, 'count': 17},
 {'year': 2024, 'dry_season_count': 10, 'wet_season_count': 0, 'count': 11},
 {'year': 2025, 'dry_season_count': 10, 'wet_season_count': 0, 'count': 10}]

### Analysis Plan Framework
- Clouds on 20% Threshold 
- Recent Data will be used ( 2025 vs 2019 ) in the same periods or full year ( 2024 vs 2019 ) or last 3 years changes.

In [ ]:
def setting_up_analysis_periods( sar_yearly_stats , optical_yearly_stats, baseline_year = None, current_year = None , use_pairing = False, custom_filter = None): 
    default_filters = {
        # SAR Images Thresholds
        'min_sar_images': 40
        # Optical Images Thresholds
        , 'min_optical_images': 5
        # Pairing Thresholds
        , 'require_data_pairing': use_pairing
        # Minimum Paired Images
        , 'min_paired_images' : 3
        # Date Tolerance in Days for Pairing
        , 'date_tolerance_days': 3
        # Use the Recent Years More Preferentially
        , 'prefer_recent_years' : True 
        # Set the the intermediate years to be at least 3 years apart
        , 'min_years_of_intermediate' : 3
    }

    # Override the custom filters if provided  
    if custom_filter:
        default_filters.update(custom_filter)
    
    # Extraction Configuration 
    print(f"\nConfiguration:")
    print(f"Pairing Mode: {'STRICT (Paired data required)' if use_pairing else 'HYBRID (All data used)'}")
    print(f"Min SAR Images: {default_filters['min_sar_images']}")
    print(f"Min Optical Images: {default_filters['min_optical_images']}")
    if use_pairing:
        print(f"Min Paired Images: {default_filters['min_paired_images']}")
    print(f"Date Tolerance: {default_filters['date_tolerance_days']} days\n")

    # Optical Image dictionary 
    optical_dict = {year['year'] : year for year in optical_yearly_stats}

    # Evaluate the good Years and All years 
    all_years_data = []
    good_years_data = []

    for sar_year in sar_yearly_stats:
        year_num = sar_year['year']
        sar_count = sar_year['count'] 

        # print(sar_count)

        # Firstly check if the year exists in optical data
        if year_num not in optical_dict:
            print(f"{year_num} : No optical data available")
            continue
        
        optical_count = optical_dict[year_num]['count']

        # Calculate the paired images 
        paired_count = min(sar_count, optical_count) if use_pairing else 0

        # Apply the filtering criteria 
        sar_pass = sar_count >= default_filters['min_sar_images'] 
        optical_pass = optical_count >= default_filters['min_optical_images']
        pairing_pass = paired_count >= default_filters['min_paired_images'] if use_pairing else True
    
        # Create combined data entry 
        year_data = {
            'year': year_num
            , 'sar_data' : sar_year 
            , 'optical_data' : optical_dict[year_num]
            , 'sar_count' : sar_count
            , 'optical_count' : optical_count
            , 'paired_count' : paired_count
            , 'total_count' : sar_count + optical_count
            , 'passed_sar_filter' : sar_pass
            , 'passed_optical_filter' : optical_pass
            , 'passes_pairing_filter' : pairing_pass
            , 'passes_all_filters' : sar_pass and optical_pass and pairing_pass
        }

        all_years_data.append(year_data) 

        # Display evaluation results 
        if year_data['passes_all_filters']:
            if use_pairing:
                print(f"{year_num} : PASS (SAR: {sar_count}, Optical: {optical_count}, Paired: {paired_count})")
            else: 
                print(f"{year_num} : PASS (SAR: {sar_count}, Optical: {optical_count})")
            good_years_data.append(year_data) 
        else:
            fail_reasons = []
            if not sar_pass:
                fail_reasons.append(f"SAR images ({sar_count} < {default_filters['min_sar_images']})")
            if not optical_pass:
                fail_reasons.append(f"Optical images ({optical_count} < {default_filters['min_optical_images']})")
            if use_pairing and not pairing_pass:
                fail_reasons.append(f"Paired images ({paired_count} < {default_filters['min_paired_images']})")
            print(f"{year_num} : FAIL ({', '.join(fail_reasons)})")

        # Relaxation Criteria if there are no good years in previous method 
    if len(good_years_data) == 0 : 
        print("\nNo good years found with current criteria. Relaxed the criteria\n")
        relaxed_filters = (
            'min_sar_images' : 20
            , 'min_optical_images' : 3
            , 'min_paired_images' : 2
        )


    print(f"\nFound {len(good_years_data)} good years out of {len(all_years_data)} total years.")    



In [70]:
# Basic usage with your existing variables
setting_up_analysis_periods(
    sar_yearly_summary, 
    optical_yearly_summary
)


Configuration:
Pairing Mode: HYBRID (All data used)
Min SAR Images: 40
Min Optical Images: 5
Date Tolerance: 3 days

2014 : No optical data available
2015 : No optical data available
2016 : No optical data available
2017 : No optical data available
2018 : FAIL (Optical images (0 < 5))
2019 : PASS (SAR: 57, Optical: 17)
2020 : PASS (SAR: 49, Optical: 11)
2021 : PASS (SAR: 46, Optical: 10)
2022 : FAIL (Optical images (2 < 5))
2023 : FAIL (SAR images (39 < 40))
2024 : FAIL (SAR images (34 < 40))
2025 : FAIL (SAR images (26 < 40))

Found 3 good years out of 8 total years.


In [ ]:
def setting_up_analysis_periods_testing(sar_yearly_stats, optical_yearly_stats, baseline_year=None, current_year=None, use_pairing=False, custom_filters=None):
    default_filters = {
        # SAR Images Thresholds
        'min_sar_images': 40
        # Optical Images Thresholds
        , 'min_optical_images': 5
        # Pairing Thresholds
        , 'require_data_pairing': use_pairing
        # Minimum Paired Images
        , 'min_paired_images' : 3
        # Date Tolerance in Days for Pairing
        , 'date_tolerance_days': 7
        # Use the Recent Years More Preferentially
        , 'prefer_recent_years' : True 
        # Set the the intermediate years to be at least 3 years apart
        , 'min_years_of_intermediate' : 3
    }
    
    # Override with custom filters if provided
    if custom_filters:
        default_filters.update(custom_filters)
    
    print(f"\nConfiguration:")
    print(f"  Pairing Mode: {'STRICT (Paired data required)' if use_pairing else 'HYBRID (All data used)'}")
    print(f"  Min SAR Images: {default_filters['min_sar_images']}")
    print(f"  Min Optical Images: {default_filters['min_optical_images']}")
    if use_pairing:
        print(f"  Min Paired Images: {default_filters['min_paired_images']}")
    print(f"  Date Tolerance: ±{default_filters['date_tolerance_days']} days")
    
    # Create optical lookup dictionary
    optical_dict = {year['year']: year for year in optical_yearly_stats}
    
    print(f"\nAvailable Years:")
    print(f"  SAR: {[year['year'] for year in sar_yearly_stats]}")
    print(f"  Optical: {[year['year'] for year in optical_yearly_stats]}")
    
    # Evaluate each year
    all_years_data = []
    good_years = []
    
    print(f"\nEvaluating Years:")
    for sar_year in sar_yearly_stats:
        year_num = sar_year['year']
        sar_count = sar_year['count']
        
        # Check if optical data exists
        if year_num not in optical_dict:
            print(f"  {year_num}: ❌ No optical data available")
            continue
            
        optical_count = optical_dict[year_num]['count']
        
        # Calculate paired images (simplified - use minimum for now)
        paired_count = min(sar_count, optical_count) if use_pairing else 0
        
        # Apply filters
        sar_ok = sar_count >= default_filters['min_sar_images']
        optical_ok = optical_count >= default_filters['min_optical_images']
        paired_ok = paired_count >= default_filters['min_paired_images'] if use_pairing else True
        
        # Create year data entry
        year_data = {
            'year': year_num,
            'sar_data': sar_year,
            'optical_data': optical_dict[year_num],
            'sar_count': sar_count,
            'optical_count': optical_count,
            'paired_count': paired_count,
            'total_images': sar_count + optical_count,
            'passes_sar_filter': sar_ok,
            'passes_optical_filter': optical_ok,
            'passes_paired_filter': paired_ok,
            'passes_all_filters': sar_ok and optical_ok and paired_ok
        }
        
        all_years_data.append(year_data)
        
        # Display evaluation results
        if year_data['passes_all_filters']:
            if use_pairing:
                print(f"  {year_num}: ✅ SAR={sar_count}, Optical={optical_count}, Paired={paired_count}")
            else:
                print(f"  {year_num}: ✅ SAR={sar_count}, Optical={optical_count}")
            good_years.append(year_data)
        else:
            reasons = []
            if not sar_ok:
                reasons.append(f"SAR={sar_count}<{default_filters['min_sar_images']}")
            if not optical_ok:
                reasons.append(f"Optical={optical_count}<{default_filters['min_optical_images']}")
            if use_pairing and not paired_ok:
                reasons.append(f"Paired={paired_count}<{default_filters['min_paired_images']}")
            print(f"  {year_num}: ❌ {', '.join(reasons)}")
    
    print(f"\nFound {len(good_years)} years meeting criteria")